In [1]:
import sys
import os
import pytorch_lightning as pl
from src.videogpt.vqvae import VQVAE
from pytorch_lightning.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import torch
from torchvision.io import read_video, read_video_timestamps
import math
import torch.nn.functional as F

In [19]:
model = VQVAE.load_from_checkpoint("/home/sana4471/projects/rrg-pbellec/sana4471/movie_decoding_sa/shinobi_data_set/code_100K/model/Shinobi_VQVAE/lightning_logs/version_15337399/checkpoints/epoch=195-step=156603.ckpt")
model

VQVAE(
  (encoder): Encoder(
    (convs): ModuleList(
      (0): SamePadConv3d(
        (conv): Conv3d(3, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
      )
      (1): SamePadConv3d(
        (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
      )
      (2): SamePadConv3d(
        (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
      )
      (3): SamePadConv3d(
        (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
      )
    )
    (conv_last): SamePadConv3d(
      (conv): Conv3d(240, 240, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    )
    (res_stack): Sequential(
      (0): AttentionResidualBlock(
        (block): Sequential(
          (0): BatchNorm3d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): SamePadConv3d(
            (conv): Conv3d(240, 120, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
          )
          (3): BatchNorm3d(120, eps=1e-05, momentum=0.1, 

In [20]:
import h5py
A=h5py.File('/home/sana4471/projects/rrg-pbellec/sana4471/movie_decoding_sa/shinobi_data_set/data/shinobi_frames_128.hdf5', 'r')
B=A['sub-04']
C=B['Level4']
Data=C['sub-04_ses-001_task-shinobi_run-04_level-4_rep-04']

In [21]:
C.keys()

<KeysViewHDF5 ['sub-04_ses-001_task-shinobi_run-01_level-4_rep-01', 'sub-04_ses-001_task-shinobi_run-02_level-4_rep-02', 'sub-04_ses-001_task-shinobi_run-03_level-4_rep-03', 'sub-04_ses-001_task-shinobi_run-04_level-4_rep-04', 'sub-04_ses-001_task-shinobi_run-05_level-4_rep-05', 'sub-04_ses-002_task-shinobi_run-01_level-4_rep-01', 'sub-04_ses-002_task-shinobi_run-02_level-4_rep-02', 'sub-04_ses-002_task-shinobi_run-03_level-4_rep-03', 'sub-04_ses-002_task-shinobi_run-04_level-4_rep-04', 'sub-04_ses-002_task-shinobi_run-05_level-4_rep-05', 'sub-04_ses-003_task-shinobi_run-01_level-4_rep-01', 'sub-04_ses-003_task-shinobi_run-02_level-4_rep-02', 'sub-04_ses-003_task-shinobi_run-03_level-4_rep-03', 'sub-04_ses-003_task-shinobi_run-04_level-4_rep-04', 'sub-04_ses-003_task-shinobi_run-05_level-4_rep-05', 'sub-04_ses-004_task-shinobi_run-01_level-4_rep-01', 'sub-04_ses-004_task-shinobi_run-02_level-4_rep-02', 'sub-04_ses-004_task-shinobi_run-03_level-4_rep-03', 'sub-04_ses-004_task-shinobi_ru

In [22]:
Data2=Data[280:296,:,:,:]
print(Data2.shape, type(Data2))

(16, 3, 128, 128) <class 'numpy.ndarray'>


In [23]:
Data2.shape

(16, 3, 128, 128)

In [24]:
import torch
import numpy as np
x_np= Data2
# transpose the numpy array to shape (100, 128, 128, 3)
x_np = x_np.transpose((0, 2, 3, 1))
# convert the numpy array to pytorch tensor
x_torch = torch.from_numpy(x_np)
# print the shape of pytorch tensor
print(x_torch.shape)

torch.Size([16, 128, 128, 3])


In [25]:
def preprocess(video, resolution, sequence_length=None):
    # video: THWC, {0, ..., 255}
    video = video.permute(0, 3, 1, 2).float() / 255. # TCHW
    t, c, h, w = video.shape

    # temporal crop
    if sequence_length is not None:
        assert sequence_length <= t
        video = video[:sequence_length]

    # scale shorter side to resolution
    scale = resolution / min(h, w)
    if h < w:
        target_size = (resolution, math.ceil(w * scale))
    else:
        target_size = (math.ceil(h * scale), resolution)
    video = F.interpolate(video, size=target_size, mode='bilinear',
                          align_corners=False)

    # center crop
    t, c, h, w = video.shape
    w_start = (w - resolution) // 2
    h_start = (h - resolution) // 2
    video = video[:, :, h_start:h_start + resolution, w_start:w_start + resolution]
    video = video.permute(1, 0, 2, 3).contiguous() # CTHW

    video -= 0.5

    return video

In [26]:
sub_video1_2 = preprocess(x_torch, 128, 16).unsqueeze(0)

In [27]:
sub_video1_2.shape

torch.Size([1, 3, 16, 128, 128])

In [28]:
sub_video=sub_video1_2
with torch.no_grad():
    vqvae=model
    encodings = vqvae.encode(sub_video)
    video_recon = vqvae.decode(encodings)
    video_recon = torch.clamp(video_recon, -0.5, 0.5)

In [29]:
 video_recon.shape

torch.Size([1, 3, 16, 128, 128])

In [30]:
'''
videos = torch.cat((sub_video1_2, video_recon), dim=-1)
videos = videos[0].permute(1, 2, 3, 0) # CTHW -> THWC
videos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')

fig = plt.figure()
plt.title('real (left), reconstruction (right)')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())
'''

"\nvideos = torch.cat((sub_video1_2, video_recon), dim=-1)\nvideos = videos[0].permute(1, 2, 3, 0) # CTHW -> THWC\nvideos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')\n\nfig = plt.figure()\nplt.title('real (left), reconstruction (right)')\nplt.axis('off')\nim = plt.imshow(videos[0, :, :, :])\nplt.close()\n\ndef init():\n    im.set_data(videos[0, :, :, :])\n\ndef animate(i):\n    im.set_data(videos[i, :, :, :])\n    return im\n\nanim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps\nHTML(anim.to_html5_video())\n"

In [31]:
from src.videogpt.gpt import VideoGPT
gpt = VideoGPT.load_from_checkpoint("/home/sana4471/projects/rrg-pbellec/sana4471/movie_decoding_sa/shinobi_data_set/code_6M/model/shinobi_GPT/lightning_logs/version_48027631/checkpoints/best_recon_epoch=79-step=115999.ckpt")
gpt

VideoGPT(
  (vqvae): VQVAE(
    (encoder): Encoder(
      (convs): ModuleList(
        (0): SamePadConv3d(
          (conv): Conv3d(3, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
        )
        (1): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
        )
        (2): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
        )
        (3): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
        )
      )
      (conv_last): SamePadConv3d(
        (conv): Conv3d(240, 240, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      )
      (res_stack): Sequential(
        (0): AttentionResidualBlock(
          (block): Sequential(
            (0): BatchNorm3d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
            (2): SamePadConv3d(
              (conv): Conv3d(240, 120, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False

In [32]:
gpt.eval()

VideoGPT(
  (vqvae): VQVAE(
    (encoder): Encoder(
      (convs): ModuleList(
        (0): SamePadConv3d(
          (conv): Conv3d(3, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
        )
        (1): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(2, 2, 2))
        )
        (2): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
        )
        (3): SamePadConv3d(
          (conv): Conv3d(240, 240, kernel_size=(4, 4, 4), stride=(1, 2, 2))
        )
      )
      (conv_last): SamePadConv3d(
        (conv): Conv3d(240, 240, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      )
      (res_stack): Sequential(
        (0): AttentionResidualBlock(
          (block): Sequential(
            (0): BatchNorm3d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
            (2): SamePadConv3d(
              (conv): Conv3d(240, 120, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False

In [33]:
samples = gpt.sample(n=1, batch=sub_video1_2)

100%|██████████| 256/256 [00:16<00:00, 16.00it/s]


In [34]:
samples.shape

torch.Size([1, 3, 16, 128, 128])

In [35]:
videos = torch.cat((sub_video1_2, samples), dim=-1)
videos = videos[0].permute(1, 2, 3, 0) # CTHW -> THWC
videos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')

fig = plt.figure()
plt.title('real (left), Predicted_frames(right)')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())